In [1]:
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script
from io import BytesIO

In [2]:
from ecc import PrivateKey, S256Point, Signature
from helper import decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160, h160_to_p2sh_address
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, TxFetcher
from urllib.request import Request, urlopen
import json

In [3]:
privkey_1 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","big"))
my_address_1 = privkey_1.point.address(testnet=True)
print(my_address_1)

mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8


In [4]:
privkey_2 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","little"))
my_address_2 = privkey_2.point.address(testnet=True)
print(my_address_2)

mwuCRN8bMLWN9ETs29U8KseDkau72M9byF


# Let's try to create a multisignature address that needs both signatures from privkey_1 and privkey_2

In [5]:
m=2
op_m = 82
n=2
op_n=82
length_pubkey=33
pubkey1=privkey_1.point.sec()
pubkey2=privkey_2.point.sec()

In [6]:
pubkey1.hex()

'03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4'

In [7]:
pubkey2.hex()

'031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee'

In [8]:
#redeem_script = Script([op_m, length_pubkey, pubkey1, length_pubkey, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script = Script([op_m, pubkey1, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [9]:
serialized_redeem = redeem_script.serialize()
print(serialized_redeem.hex())
serialized_redeem_raw=serialized_redeem[1:]
print(serialized_redeem_raw.hex())

47522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae
522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae


In [10]:
multi_sig_addr = h160_to_p2sh_address(hash160(serialized_redeem_raw), testnet=True)
multi_sig_addr

'2NAqp3V17G4iBekEwWuYkgzoEyWiULhYZLs'

# Now let's try to send some BTCs to this address

In [11]:
def get_index(outs_list, address):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(address) or out.script_pubkey.cmds[1] == decode_base58(address):
            return index
    return "output index not found"


def get_amount_utxo(outs_list, index):
    return outs_list[index].amount


def get_tx_ins_utxo(prev_tx_id_list, receiving_address, testnet=True):
    
    tx_ins = []
    
    for prev_tx_id in prev_tx_id_list:
        prev_tx = TxFetcher.fetch(prev_tx_id, testnet)
        print(prev_tx)
        prev_index = get_index(prev_tx.tx_outs, receiving_address)
        tx_in = TxIn(bytes.fromhex(prev_tx_id),prev_index)
        utxo = get_amount_utxo(prev_tx.tx_outs, prev_index)
        print(f"index 1: {prev_index}, amount: {utxo}")
        tx_ins.append({"tx_in": tx_in, "utxo": utxo})
        
    return tx_ins

def calculate_fee(version, tx_ins, tx_outs, locktime, privkey, redeem_script, testnet=True, multisig =False, fee_per_byte = 8 ):
    """
    privkey: can be just one or a list of private keys in the case of multisignature.
    """
    tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
    print(tx)

    # sign the inputs in the transaction object using the private key
    if multisig:
        for tx_input in range(len(tx_ins)):
            print(my_multisig_tx.sign_input_multisig(tx_input, privkey, redeem_script))
    else:
        for tx_input in range(len(tx_ins)):
            print(tx.sign_input(tx_input, privkey))
        # print the transaction's serialization in hex

    #Let's calculate the fee and the change:
    tx_size = len(my_tx.serialize().hex())
    #fee_per_byte = 8 # I changed from 2 to 10 after sending this transaction because it had really low appeal to miners.
    fee = tx_size * fee_per_byte
    print(f"fee: {fee}")
    return fee

def calculate_change(utxo_list, fee, amountTx):
    total_utxo = sum(utxo_list)
    total_out = sum(amountTx)
    change = total_utxo - fee - total_out
    print(f"change {change}")
    if change < 0:
        return "Not enough utxos"
    #Let's make sure that we are actually spending the exact amount of the UTXO
    total_send=fee+total_out+change
    diff = total_utxo-total_send
    print(f"total {total_send}, diff: {diff}")
    return change

    

In [12]:
from script import p2sh_script
# Let's build the tx:
tx_outs =[]
tx_ins =[]

change = int(0.01 * 100000000)# we are going to fix this later
amountTx= int(0.01 * 100000000)

#It will send 0.01 to the multisig address
output = TxOut(amountTx, p2sh_script(decode_base58(multi_sig_addr)))
print(output)
tx_outs.append(output)

change_output = TxOut(change, p2pkh_script(decode_base58(my_address_1)))
print(change_output)
tx_outs.append(change_output)

#block explorer: https://testnet.smartbit.com.au/address/mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8/unspent

prev_tx_id_list = ["2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18",
                  "67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5"]

tx_ins_utxo = get_tx_ins_utxo(prev_tx_id_list, my_address_1, testnet=True)
    
tx_ins = [x["tx_in"] for x in tx_ins_utxo]
print(tx_ins)

utxos = [x["utxo"] for x in tx_ins_utxo]
print(utxos)

my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_tx)

fee = calculate_fee(1, tx_ins, tx_outs, 0, privkey=privkey_1, redeem_script=None, testnet=True)
change = calculate_change(utxos, fee, [amountTx])

#Let's modify the amount in the change_output:
my_tx.tx_outs[-1].amount = change

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))

print(my_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))

1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18
version: 1
tx_ins:
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx_outs:
980544:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0, amount: 980544
tx: 67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:1
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
591336:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 1, amount: 591336
[2f9c289966440d088978f9c674a0e596d47db5a91d6d5fc4df7ed38aee0a2d18:0, 67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5:1]
[980544, 591336]
tx: 1125840de36407c691

In [13]:
my_tx.verify_input(0)

OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG

combined script[b'0E\x02!\x00\x81`=\xb2N)\x8e\x83i\x97>\xed\xd1\x10f\x06\xdfE\xc3\xcc\x13\xedvp\x03\xbe\xfc^\xb4\xdd\xf0\xa2\x02 p,4\xdag\xac<\xaf\xb3\x14\xe097\xf4\xde\xdc\xe9\xf5\xa1\x01\x86u\x1bK\xd5z\x0eY\x11\xe3\n\xc5\x01', b"\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4", 118, 169, b'R\x90>\xfc\x10\x04\xde\x01\x88;\xa3h{\xe2\xa8\xeaOk\x1b\x19', 136, 172]



True

# Now let's try to spend the UTXO from the multi-sig address

In [14]:
reload(tx)

<module 'tx' from '/Users/oscareduardosernarosero/Documents/US/Rice U/blockchain/bitcoin/basics/final/tx.py'>

In [15]:
###### tx_outs =[]
tx_ins =[]
tx_outs =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)

prev_tx_id_list = ["7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811"]

tx_ins_utxo = get_tx_ins_utxo(prev_tx_id_list, multi_sig_addr, testnet=True)
    
tx_ins = [x["tx_in"] for x in tx_ins_utxo]
print(tx_ins)

utxos = [x["utxo"] for x in tx_ins_utxo]
print(utxos)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)

fee = calculate_fee(1, tx_ins, tx_outs, 0, privkey=[privkey_1, privkey_2], 
                    redeem_script=redeem_script, testnet=True, multisig=True, fee_per_byte = 12)

amountTx = sum(utxos) - fee

#Let's modify the amount in the change_output:
my_multisig_tx.tx_outs[0].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier

print(my_multisig_tx.serialize().hex())



1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811
version: 1
tx_ins:
02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951:0
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1593566:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0, amount: 1000000
[7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:0]
[1000000]
tx: 6859670602077ba2aec901d5817421e3926885c21247f48578fe40a6662f16b7
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:0
tx_outs:
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
tx: 6859670602077ba2aec901d5817421e3926885c21247f48578fe40a6662f16b7
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:0
tx_outs:
1000000:OP_DUP OP_HASH160 

In [16]:
my_multisig_tx.verify()

OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
scriptPubKey: OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
commands redeemscript: 522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae
redeem_script present
script: OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

combined script[0, b'0E\x02!\x00\xb7>\x8f\x0e{}(\x80D\x80u\x15\xb7\xc8\x9aY\x9fA\xad\xe4i\x14&\xa9\xaf\x95\xad\x83\xeaQK\x9a\x02 >\xb1\\(\xd0\xa8\xcf\x88\xb6\x1fP2\x1b\x98\xdb\xf2\x9a\xa2fP\x19\xf7\xaa\x8e\xb1\xfe\xb0\x07\xd2\xf0\x93S\x01', b"0D\x02 &#\xb5:p\xac&\xaf\x1f\xe3\x98\x1cB_b%k\xfe\x12\x8c\x07\xfe=\xf1\x9f\xb5\xbe4$\\,\x85\x02 PD\xd9\x14\xb2R\xdc\xea\x97\x869x\xec\xde\xae\xbf\xab.'\x1a\xe8\xc8\xa9\x8f,\xa1\x933\x83i\xef\x1b\x01", b"R!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\

True

In [17]:
my_multisig_tx

tx: 4b477c6b2009d5be88e80455cd74b19d9fc441f0e75613ed95080a723cfd8500
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:0
tx_outs:
991096:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0

In [18]:
rebuilt_script = Script.parse(BytesIO(bytes.fromhex("47522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae")))
rebuilt_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [19]:
for cmd in my_multisig_tx.tx_ins[0].script_sig.cmds:
    try: print(cmd.hex())
    except: print(cmd)

0
3045022100b73e8f0e7b7d288044807515b7c89a599f41ade4691426a9af95ad83ea514b9a02203eb15c28d0a8cf88b61f50321b98dbf29aa2665019f7aa8eb1feb007d2f0935301
304402202623b53a70ac26af1fe3981c425f62256bfe128c07fe3df19fb5be34245c2c8502205044d914b252dcea97863978ecdeaebfab2e271ae8c8a98f2ca193338369ef1b01
522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae


In [20]:
#my_multisig_tx.tx_outs.pop(0)

In [21]:
#upsidedown = Script(my_multisig_tx.tx_ins[0].script_sig.cmds[::-1])
straight = Script(my_multisig_tx.tx_ins[0].script_sig.cmds)
rebuilt_script2 = Script(rebuilt_script.cmds[1:])
#upsidedown.cmds.pop(0)
straight.cmds.pop(-1)
print(straight)

OP_0 3045022100b73e8f0e7b7d288044807515b7c89a599f41ade4691426a9af95ad83ea514b9a02203eb15c28d0a8cf88b61f50321b98dbf29aa2665019f7aa8eb1feb007d2f0935301 304402202623b53a70ac26af1fe3981c425f62256bfe128c07fe3df19fb5be34245c2c8502205044d914b252dcea97863978ecdeaebfab2e271ae8c8a98f2ca193338369ef1b01


In [22]:
combined_part2 = straight+ rebuilt_script

In [23]:
from helper import hash256
raw_modified=bytes.fromhex("0100000001117892d7a31c823c437579c6d3f915a2841280aba7e026df88017d36fc0bc47d0000000047522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52aeffffffff01caf90e00000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac0000000001000000")
raw_script_sig="da00473044022028ddc64e8f07f5f8686b8290de3a72f855761b615af87c93c122e2bf20a367c802206abf030c6f956e9d0e508cbb79cd15beed1f406f06de34b93932b56d987b39e001483045022100851ef022b4e3a5afd2af468093564f4b93a6c53d1700fb90c97c283ec6cdbde802200b5eda1f94e48e89dcae9b5e0441f388db6d8141069f0581e1063bd488b450bb0147522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae"
z=int.from_bytes(hash256(raw_modified),"big")


In [25]:
combined_part2.evaluate(z, None)

False

In [26]:
combined_part2

OP_0 3045022100b73e8f0e7b7d288044807515b7c89a599f41ade4691426a9af95ad83ea514b9a02203eb15c28d0a8cf88b61f50321b98dbf29aa2665019f7aa8eb1feb007d2f0935301 304402202623b53a70ac26af1fe3981c425f62256bfe128c07fe3df19fb5be34245c2c8502205044d914b252dcea97863978ecdeaebfab2e271ae8c8a98f2ca193338369ef1b01 OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [27]:

sec1=bytes.fromhex(str(combined_part2.cmds[4].hex()))
sec2=bytes.fromhex(str(combined_part2.cmds[5].hex()))
der1=bytes.fromhex(str(combined_part2.cmds[1][:-1].hex()))
der2=bytes.fromhex(str(combined_part2.cmds[2][:-1].hex()))
point1=S256Point.parse(sec1)
point2=S256Point.parse(sec2)
sig1=Signature.parse(der1)
sig2=Signature.parse(der2)
print(point1.verify(z,sig1))
print(point2.verify(z,sig2))

False
False


# Let's try with a lower fee

In [28]:
tx_ins =[]
tx_outs =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)

prev_tx_id_list = ["67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5"]

tx_ins_utxo = get_tx_ins_utxo(prev_tx_id_list, multi_sig_addr, testnet=True)
    
tx_ins = [x["tx_in"] for x in tx_ins_utxo]
print(tx_ins)

utxos = [x["utxo"] for x in tx_ins_utxo]
print(utxos)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)

fee = calculate_fee(1, tx_ins, tx_outs, 0, privkey=[privkey_1, privkey_2], 
                    redeem_script=redeem_script, testnet=True, multisig=True, fee_per_byte = 12)

amountTx = sum(utxos) - fee

#Let's modify the amount in the change_output:
my_multisig_tx.tx_outs[0].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier

print(my_multisig_tx.serialize().hex())

1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:1
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
591336:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0, amount: 1000000
[67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5:0]
[1000000]
tx: 51fa79324d955e2559bd745e49360bbabd2cb563c0b8775493b41a93b2bff2e2
version: 1
tx_ins:
67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5:0
tx_outs:
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
tx: 51fa79324d955e2559bd745e49360bbabd2cb563c0b8775493b41a93b2bff2e2
version: 1
tx_ins:
67d3baf44adefc13c8635f51e45e6c1de6fa47cf18f4afbdc05ab0c95bd898e5:0
tx_outs:
1000000:OP_DUP OP_HASH160 5

# A fee of 12 satoshis per byte has a "Medium" preference by miners according to the block explorer Blockcypher.com

In [30]:
type('n')

str